In [48]:
import numpy as np
import pandas as pd

df = pd.read_csv(r'./train.csv')
pd.options.display.max_columns = 999
df_corr = pd.DataFrame(abs(df.corr()['SalePrice']).sort_values(ascending=False)).index
cols = list(df_corr[:2]) + list(df_corr[-2:])

X = df[cols].iloc[:100].drop('BsmtFinSF2', axis=1)
y = X.pop('SalePrice')

X = X.values
y = y.values

In [81]:
np.random.seed(1)
class NN:
  def __init__(self, X, y, hidden_layers, loss_function='mae'):
    self.X = np.array(X)
    self.y = np.array(y)
    self.y = self.y.reshape(self.y.shape[0], 1)
    self.schema = [X.shape[1]] + hidden_layers + [self.y.shape[1]]
    self.schema_len = range(len(self.schema[:-1]))
    self.loss_function = loss_function
    for i, layer in enumerate(self.schema[:-1]):
      setattr(self, f'W{i}', np.random.uniform(-1, 1, (layer, self.schema[i+1])))

      
#   # sigmoid
#   def nonlin(self, x, deriv=False):
#     if(deriv==True):
#       return x * (1 - x)
#     return 1/ (1 + np.exp(-x))
  
#   # ReLU
#   def nonlin(self, x, deriv=False):
#     if(deriv==True):
#       return np.where(x > 0, 1, 0)
#     return np.maximum(0, x)
  
  # leaky ReLU
  def nonlin(self, x, deriv=False):
    if(deriv==True):
      return np.where(x > 0, 1, x/100)
    return np.where(x > 0, x, 1/100)
  
  
  
  def forward(self, X=None):
    if X:
      self.l0 = np.array(X)
    else:
      self.l0 = self.X
    for i in self.schema_len:
      l, W = (getattr(self, j) for j in f'l{i} W{i}'.split())
      setattr(self, f'l{i+1}', self.nonlin(l.dot(W)))
    return getattr(self, f'l{len(self.schema_len)}')
    
  
  def loss(self, error):
    loss_dict = {
      'mse': lambda x: np.mean(x**2),
      'mae': lambda x: np.mean(abs(x)),
    }
    loss_function = loss_dict[self.loss_function]
    return loss_function(error)
    
  def backward(self):

    # calculation loss
    for i in reversed(self.schema_len):
      i += 1
      l = getattr(self, f'l{i}')
      if i == len(self.schema_len):
        #####################################################
        error = self.y - l
        self.error = self.loss(error)
      else:
        delta, W = (getattr(self, j) for j in f'l{i}delta W{i}'.split())
        error = delta.dot(W.T)
      setattr(self, f'l{i-1}delta', error * self.nonlin(l, deriv=True))
        ######################################################
    # adjusting weights
    for i in reversed(self.schema_len):
      l, W, delta = (getattr(self, j) for j in f'l{i} W{i} l{i}delta'.split())
      setattr(self, f'W{i}', W + l.T.dot(delta))
    
    
  def train(self, epochs=1000, print_nth_epoch=100):
    for j in range(epochs):
      self.forward()
      self.backward()
      if print_nth_epoch and not j % print_nth_epoch:
        print(f'Error: {self.error}')
       
      
  def predict(self, x):
    return self.forward(x)

In [82]:
nn = NN(X, y, [2])

In [83]:
nn.train(100, 25)

Error: 173820.64999999994
Error: 173820.64999999994
Error: 173820.64999999994
Error: 173820.64999999994


In [13]:
nn.predict([X[0]])

array([[1.]])

In [14]:
nn.loss_function

'mae'